In [65]:
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.io as pio
import sklearn
import warnings
import sksurv.datasets
import numpy as np
import joblib
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import XGBClassifier

from itertools import product
from tqdm import tqdm
from xgbse import XGBSEKaplanNeighbors
from xgbse.converters import convert_to_structured
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.exceptions import UndefinedMetricWarning
from sklearn import set_config
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import ParameterGrid
from sksurv.datasets import load_breast_cancer
from sksurv.metrics import cumulative_dynamic_auc
from sksurv.metrics import concordance_index_censored
from sksurv.linear_model import CoxnetSurvivalAnalysis, CoxPHSurvivalAnalysis
from sksurv.preprocessing import OneHotEncoder
from sksurv.util import Surv


from sksurv.ensemble import GradientBoostingSurvivalAnalysis


warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
set_config(display="text")

In [77]:
df_corse=pd.read_csv('https://projet-incendie.s3.eu-west-3.amazonaws.com/dataset_modele_decompte.csv', sep=';')




In [ ]:
df_corse['Feu prévu'] = df_corse['Feu prévu'].astype(bool)
df_variables = df_corse[[
    'moyenne precipitations mois', 'moyenne temperature mois',
    'moyenne evapotranspiration mois', 'moyenne vitesse vent année',
    'moyenne vitesse vent mois', 'moyenne temperature année',
    'RR', 'UM', 'ETPMON', 'TN', 'TX', 'évènement', 'décompte', 'Nombre de feu par an',
    'Nombre de feu par mois', 'jours_sans_pluie', 'jours_TX_sup_30', 
    'compteur jours vers prochain feu', 'compteur feu carré', 'compteur feu log', 'ETPGRILLE_7j',
    'moyenne precipitations année', 'moyenne evapotranspiration année'
]]

df_variables.rename(columns={
    'RR': 'Précipitations en mm',
    'TN': 'Température minimale sous abri',
    'TX': 'Température maximale sous abri',
    'UM': 'Humidité moyenne en %',
    'ETPMON': 'Evapotranspiration en mm',
       
}, inplace=True)


#### XGBoost Régression classique

In [ ]:

threshold = 0.5
cols_to_keep = df_corse.columns[df_corse.isnull().mean() < threshold]
df_clean = df_corse[cols_to_keep].copy()

features = [
    'moyenne precipitations mois', 'moyenne temperature mois',
    'moyenne evapotranspiration mois', 'moyenne vitesse vent année',
    'moyenne vitesse vent mois', 'moyenne temperature année',
    'RR', 'UM', 'ETPMON', 'TN', 'TX', 'évènement', 'Nombre de feu par an',
    'Nombre de feu par mois', 'jours_sans_pluie', 'jours_TX_sup_30', 
    'compteur jours vers prochain feu', 'compteur feu carré', 'compteur feu log', 'ETPGRILLE_7j',
    'moyenne precipitations année', 'moyenne evapotranspiration année'
]
features = [f for f in features if f in df_clean.columns]


df_clean = df_clean.dropna(subset=["décompte"])


#### XGBoost Regressor Survival Cox

In [93]:
df_clean = df_clean.rename(columns={"Feu prévu": "event", "décompte": "duration"})

y_structured = Surv.from_dataframe("event", "duration", df_clean)

X = df_clean[features]
y = y_structured

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

event_train = y_train["event"]
duration_train = y_train["duration"]

event_test = y_test["event"]
duration_test = y_test["duration"]

pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("xgb", XGBRegressor(
        objective="survival:cox",
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        tree_method="hist",
        device="cuda",       
        random_state=42
    ))
])

pipeline.fit(X_train, duration_train, xgb__sample_weight=event_train)

risques = pipeline.predict(X_test)

cindex = concordance_index_censored(event_test, duration_test, risques)[0]
print(f"C-index test : {cindex:.3f}")


C-index test : 0.781


#### GridSearch et XGBoost

In [94]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
# from xgboost import XGBRegressor, DMatrix, train
# from sksurv.util import Surv
# from lifelines import CoxPHFitter

# # 🔹 Préparation des données réelles
# df_clean = df_clean.rename(columns={"Feu prévu": "event", "décompte": "duration"})
# y_structured = Surv.from_dataframe("event", "duration", df_clean)

# X = df_clean[features]
# y = y_structured

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# event_train = y_train["event"]
# duration_train = y_train["duration"]
# event_test = y_test["event"]
# duration_test = y_test["duration"]

# # 🔹 Pipeline XGBoost survie
# pipeline = Pipeline([
#     ("imputer", SimpleImputer(strategy="median")),
#     ("xgb", XGBRegressor(
#         objective="survival:cox",
#         n_estimators=100,
#         learning_rate=0.1,
#         max_depth=5,
#         tree_method="hist",
#         device="cuda",
#         random_state=42
#     ))
# ])
# pipeline.fit(X_train, duration_train, xgb__sample_weight=event_train)

# # 🔹 Prédictions réelles (log(HR))
# log_hr_test = pipeline.predict(X_test)

# # 🔹 Jeu factice pour estimer H0(t)
# df_fake = pd.DataFrame({
#     "duration": duration_train,
#     "event": event_train,
#     "const": 1
# })

# dtrain_fake = DMatrix(df_fake[["const"]])
# dtrain_fake.set_float_info("label", df_fake["duration"])
# dtrain_fake.set_float_info("label_lower_bound", df_fake["duration"])
# dtrain_fake.set_float_info("label_upper_bound", df_fake["duration"])
# dtrain_fake.set_float_info("weight", df_fake["event"])

# params = {
#     "objective": "survival:cox",
#     "eval_metric": "cox-nloglik",
#     "learning_rate": 0.1,
#     "max_depth": 1,
#     "verbosity": 0
# }
# bst_fake = train(params, dtrain_fake, num_boost_round=100)

# # 🔹 Estimation de H0(t) avec CoxPHFitter
# log_hr_fake = bst_fake.predict(dtrain_fake)  # valeurs constantes
# df_risque = pd.DataFrame({
#     "duration": duration_train,
#     "event": event_train,
#     "log_risque": log_hr_fake
# })
# cph = CoxPHFitter()
# # log_risque contient quasiment la même valeur pour tous les individus (normal, car le modèle XGBoost a été entraîné sur un dataset factice avec une seule constante).
# # Du coup, lifelines n’arrive pas à ajuster les coefficients, car il n’y a aucune information discriminante dans log_risque
# df_risque["log_risque"] += np.random.normal(0, 1e-4, size=len(df_risque))

# jours = [30, 60, 90, 180]

# # Extraire la fonction H0(t)
# h0 = cph.baseline_cumulative_hazard_

# # Interpolation de H0(t) pour les jours souhaités
# h0_interp = h0.reindex(jours, method='nearest')  # ou interpolation linéaire si besoin
# H0_t = h0_interp.values.flatten()  # shape (n_jours,)

# # Reprend les prédictions de risque du vrai modèle
# df_preds = pd.DataFrame({"log_risque": log_hr_test})
# log_HR = df_preds["log_risque"].values  # shape (n_individus,)

# # Calcul de S(t|x) pour chaque temps t
# S_t = np.exp(-np.outer(H0_t, np.exp(log_HR)))  # shape (n_jours, n_individus)

# # Probabilité d'un feu à l'échéance t
# probas_feu = 1 - S_t.T  # shape (n_individus, n_jours)

# # Mettre en DataFrame avec colonnes explicites
# probas_feu_df = pd.DataFrame(
#     probas_feu,
#     columns=[f"proba_{j}j" for j in jours]
# )



# print(df_risque.isna().sum())

# cph.fit(df_risque, duration_col="duration", event_col="event", show_progress=False)

# # 🔹 Calcul des fonctions de survie individuelles
# df_preds = pd.DataFrame({"log_risque": log_hr_test})
# surv_funcs = cph.predict_survival_function(df_preds)

# # 🔹 Affichage des fonctions de survie (ex : 5 premiers individus)
# plt.figure(figsize=(10, 6))
# for i in range(5):
#     surv_funcs.iloc[:, i].plot(label=f"Individu {i}")
# plt.title("Fonction de survie individuelle (probabilité de ne pas avoir de feu)")
# plt.xlabel("Temps")
# plt.ylabel("S(t) = P(pas de feu à t)")
# plt.legend()
# plt.grid(True)
# plt.show()

# # 🔹 Optionnel : calcul de la probabilité de feu
# # Par exemple à t = 10 :
# t_index = surv_funcs.index.get_indexer([10], method='nearest')[0]
# proba_feu_t10 = 1 - surv_funcs.iloc[t_index]
# print("\nProbabilité de feu à t ≈ 10 pour chaque individu :")
# print(proba_feu_t10.head())


In [95]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
# from xgboost import XGBRegressor, DMatrix, train
# from sksurv.util import Surv
# from lifelines import CoxPHFitter

# # 🔹 Préparation des données réelles
# df_clean = df_clean.rename(columns={"Feu prévu": "event", "décompte": "duration"})
# y_structured = Surv.from_dataframe("event", "duration", df_clean)

# X = df_clean[features]
# y = y_structured

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# event_train = y_train["event"]
# duration_train = y_train["duration"]
# event_test = y_test["event"]
# duration_test = y_test["duration"]

# # 🔹 Pipeline XGBoost survie
# pipeline = Pipeline([
#     ("imputer", SimpleImputer(strategy="median")),
#     ("xgb", XGBRegressor(
#         objective="survival:cox",
#         n_estimators=100,
#         learning_rate=0.1,
#         max_depth=5,
#         tree_method="hist",
#         device="cuda",
#         random_state=42
#     ))
# ])
# pipeline.fit(X_train, duration_train, xgb__sample_weight=event_train)

# # 🔹 Prédictions réelles (log(HR))
# log_hr_test = pipeline.predict(X_test)

# # 🔹 Jeu factice pour estimer H0(t)
# df_fake = pd.DataFrame({
#     "duration": duration_train,
#     "event": event_train,
#     "const": 1
# })

# dtrain_fake = DMatrix(df_fake[["const"]])
# dtrain_fake.set_float_info("label", df_fake["duration"])
# dtrain_fake.set_float_info("label_lower_bound", df_fake["duration"])
# dtrain_fake.set_float_info("label_upper_bound", df_fake["duration"])
# dtrain_fake.set_float_info("weight", df_fake["event"])

# params = {
#     "objective": "survival:cox",
#     "eval_metric": "cox-nloglik",
#     "learning_rate": 0.1,
#     "max_depth": 1,
#     "verbosity": 0
# }
# bst_fake = train(params, dtrain_fake, num_boost_round=100)

# # 🔹 Estimation de H0(t) avec CoxPHFitter
# log_hr_fake = bst_fake.predict(dtrain_fake)
# df_risque = pd.DataFrame({
#     "duration": duration_train,
#     "event": event_train,
#     "log_risque": log_hr_fake
# })
# df_risque["log_risque"] += np.random.normal(0, 1e-4, size=len(df_risque))

# # Ajustement du modèle Cox sur les données factices
# cph = CoxPHFitter()
# cph.fit(df_risque, duration_col="duration", event_col="event", show_progress=False)

# # 🔹 Estimation de H0(t) aux temps donnés
# jours = [30, 60, 90, 180]
# h0 = cph.baseline_cumulative_hazard_
# h0_interp = h0.reindex(jours, method='nearest')
# H0_t = h0_interp.values.flatten()

# # 🔹 Calcul des probabilités de feu
# log_HR = log_hr_test
# S_t = np.exp(-np.outer(H0_t, np.exp(log_HR)))
# probas_feu = 1 - S_t.T

# probas_feu_df = pd.DataFrame(
#     probas_feu,
#     columns=[f"proba_{j}j" for j in jours]
# )
# # # y_test est déjà une structure Surv
# # c_index, concordant, discordant, tied_risk, tied_time = concordance_index_censored(
# #     y_test["event"], y_test["duration"], -log_hr_test  # ⚠️ attention au signe : -log(HR)
# # )

# # print(f"\n🔎 Concordance Index (C-index) sur test : {c_index:.3f}")

# # # 🔹 Affichage des fonctions de survie
# # surv_funcs = cph.predict_survival_function(pd.DataFrame({"log_risque": log_HR}))
# # plt.figure(figsize=(10, 6))
# # for i in range(5):
# #     surv_funcs.iloc[:, i].plot(label=f"Individu {i}")
# # plt.title("Fonction de survie individuelle (probabilité de ne pas avoir de feu)")
# # plt.xlabel("Temps")
# # plt.ylabel("S(t) = P(pas de feu à t)")
# # plt.legend()
# # plt.grid(True)
# # plt.show()

# # # 🔹 Exemple : probabilité de feu à t ≈ 10 jours
# # t_index = surv_funcs.index.get_indexer([10], method='nearest')[0]
# # proba_feu_t10 = 1 - surv_funcs.iloc[t_index]
# # print("\nProbabilité de feu à t ≈ 10 pour chaque individu :")
# # print(proba_feu_t10.head())


### XGBOOST predict_cumulative_hazard_function

In [99]:

# 🔹 Préparation des données réelles
df_clean = df_clean.rename(columns={"Feu prévu": "event", "décompte": "duration"})
y_structured = Surv.from_dataframe("event", "duration", df_clean)

X = df_clean[features]
y = y_structured

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

event_train = y_train["event"]
duration_train = y_train["duration"]
event_test = y_test["event"]
duration_test = y_test["duration"]

# 🔹 Pipeline XGBoost survie
pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("xgb", XGBRegressor(
        objective="survival:cox",
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        tree_method="hist",
        device="cuda",
        random_state=42
    ))
])
pipeline.fit(X_train, duration_train, xgb__sample_weight=event_train)

# 🔹 Prédictions réelles (log(HR))
log_hr_test = pipeline.predict(X_test)

# 🔹 Jeu factice pour estimer le modèle de Cox
df_fake = pd.DataFrame({
    "duration": duration_train,
    "event": event_train,
    "const": 1
})
dtrain_fake = DMatrix(df_fake[["const"]])
dtrain_fake.set_float_info("label", df_fake["duration"])
dtrain_fake.set_float_info("label_lower_bound", df_fake["duration"])
dtrain_fake.set_float_info("label_upper_bound", df_fake["duration"])
dtrain_fake.set_float_info("weight", df_fake["event"])

params = {
    "objective": "survival:cox",
    "eval_metric": "cox-nloglik",
    "learning_rate": 0.1,
    "max_depth": 1,
    "verbosity": 0
}
bst_fake = train(params, dtrain_fake, num_boost_round=100)

log_hr_fake = bst_fake.predict(dtrain_fake)
df_risque = pd.DataFrame({
    "duration": duration_train,
    "event": event_train,
    "log_risque": log_hr_fake
})
df_risque["log_risque"] += np.random.normal(0, 1e-4, size=len(df_risque))

# 🔹 Modèle de Cox factice
cph = CoxPHFitter()
cph.fit(df_risque, duration_col="duration", event_col="event", show_progress=False)

# 🔹 Prédiction des fonctions de survie
jours = [30, 60, 90, 180]
df_preds = pd.DataFrame({"log_risque": log_hr_test})
surv_funcs = cph.predict_survival_function(df_preds, times=jours)

# 🔹 Probabilité de feu à chaque échéance
probas_feu = 1 - surv_funcs.T
probas_feu.columns = [f"proba_{j}j" for j in jours]

# # 🔹 Visualisation (ex : les 5 premières courbes)
# plt.figure(figsize=(10, 6))
# for i in range(5):
#     surv_funcs.iloc[:, i].plot(label=f"Individu {i}")
# plt.title("Fonction de survie individuelle (probabilité de ne pas avoir de feu)")
# plt.xlabel("Temps (jours)")
# plt.ylabel("S(t) = P(pas de feu à t)")
# plt.legend()
# plt.grid(True)
# plt.show()

# # 🔹 Exemple : probabilité de feu à t ≈ 10 jours
# t_index = surv_funcs.index.get_indexer([10], method='nearest')[0]
# proba_feu_t10 = 1 - surv_funcs.iloc[t_index]
# print("\nProbabilité de feu à t ≈ 10 pour chaque individu :")
# print(proba_feu_t10.head())

# 🔹 🔍 Évaluation avec le c-index
c_index = concordance_index_censored(event_test, duration_test, -log_hr_test)[0]
print(f"\nC-index (test) : {c_index:.3f}")


c:\Users\m_bar\Incendie\.venv\Lib\site-packages\lifelines\utils\__init__.py:1100: ConvergenceWarning:

Column(s) ['log_risque'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.


c:\Users\m_bar\Incendie\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning:

overflow encountered in exp




C-index (test) : 0.219


In [97]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# 🔹 Récupération des données
coords_test = df_clean.loc[X_test.index, ["latitude", "longitude"]].reset_index(drop=True)

times = [7, 30, 60, 90, 180]
surv_funcs = cph.predict_survival_function(df_preds, times=times)
probas_feu = 1 - surv_funcs.T
probas_feu.columns = [f"proba_{t}j" for t in times]

df_map = pd.concat([coords_test, probas_feu.reset_index(drop=True)], axis=1)

# 🔹 Fusion coordonnées similaires (moyenne si plusieurs lignes au même endroit)
df_visu = df_map.groupby(["latitude", "longitude"]).mean().reset_index()

# 🔹 Création des traces pour chaque période
fig = go.Figure()

for t in times:
    visible = True if t == 30 else False  # Seule la première période est visible au début
    fig.add_trace(go.Scattermap(
        lat=df_visu["latitude"],
        lon=df_visu["longitude"],
        mode='markers',
        marker=go.scattermap.Marker(
            size=10,
            color=df_visu[f"proba_{t}j"],
            colorscale="RdYlGn_r",
            cmin=0, cmax=1,
            colorbar=dict(title="Proba feu", title_side="top") if t == 30 else None
        ),
        text=[f"🔥 Proba feu à {t}j : {p:.1%}" for p in df_visu[f"proba_{t}j"]],
        name=f"{t} jours",
        visible=visible
    ))

# 🔹 Menu déroulant
fig.update_layout(
    updatemenus=[{
        "buttons": [
            {
                "label": f"{t} jours",
                "method": "update",
                "args": [
                    {"visible": [i == j for i in range(len(times))]},
                    {"title": f"Carte des probabilités de feu à {t} jours"}
                ]
            }
            for j, t in enumerate(times)
        ],
        "direction": "down",
        "showactive": True,
        "x": 0.01,
        "xanchor": "left",
        "y": 1.05,
        "yanchor": "top"
    }]
)

# 🔹 Autres paramètres de la carte
fig.update_layout(
    mapbox_style="open-street-map",
    mapbox_zoom=6,
    mapbox_center={"lat": df_visu["latitude"].mean(), "lon": df_visu["longitude"].mean()},
    margin={"r":0, "t":40, "l":0, "b":0},
    title="Carte des probabilités de feu à 30 jours"
)

fig.show()


c:\Users\m_bar\Incendie\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning:

overflow encountered in exp

